In [ ]:
import mysql.connector
from werkzeug.security import generate_password_hash, check_password_hash
from datetime import datetime
import json

# Mock config for testing (replace with your actual config)
class Config:
    MYSQL_HOST = 'localhost'
    MYSQL_USER = 'root'
    MYSQL_PASSWORD = '123456'
    MYSQL_DATABASE = 'zenspace_db'

print("✅ Imports successful!")
print(f"📅 Current time: {datetime.now()}")


✅ Imports successful!
📅 Current time: 2025-09-30 23:48:50.517525


In [19]:
# Replace the placeholders with your actual DB access info here
class Config:
    MYSQL_HOST = 'localhost'         # e.g., 'localhost'
    MYSQL_USER = 'root'         # e.g., 'root'
    MYSQL_PASSWORD = 'Aayush@0909' # e.g., 'secret123'
    MYSQL_DATABASE = 'zenspace_db'  # e.g., 'ZenSpaceAI'

    JWT_ACCESS_TOKEN_EXPIRES = 3600        # token expiry in seconds
    UPLOAD_FOLDER = 'static/uploads'       # uploads directory


In [20]:
def get_db_connection():
    """Create and return a database connection"""
    try:
        connection = mysql.connector.connect(
            host=Config.MYSQL_HOST,
            user=Config.MYSQL_USER,
            password=Config.MYSQL_PASSWORD,
            database=Config.MYSQL_DATABASE,
            charset='utf8mb4',
            collation='utf8mb4_unicode_ci'
        )
        print("✅ Database connection successful!")
        return connection
    except mysql.connector.Error as err:
        print(f"❌ Error connecting to MySQL: {err}")
        return None

# Test the connection
test_connection = get_db_connection()
if test_connection:
    print(f"🔗 Connected to database: {Config.MYSQL_DATABASE}")
    test_connection.close()


✅ Database connection successful!
🔗 Connected to database: zenspace_db


In [21]:
class User:
    @staticmethod
    def create_user(username, email, password):
        connection = get_db_connection()
        if not connection:
            return None
        cursor = connection.cursor()
        password_hash = generate_password_hash(password)
        try:
            query = """
                INSERT INTO users (username, email, password_hash, created_at) VALUES (%s, %s, %s, %s)
            """
            cursor.execute(query, (username, email, password_hash, datetime.now()))
            connection.commit()
            print(f"✅ User created with id {cursor.lastrowid}")
            return cursor.lastrowid
        except mysql.connector.IntegrityError as e:
            print(f"❌ User creation failed: {e}")
            return None
        finally:
            cursor.close()
            connection.close()

    @staticmethod
    def get_user_by_email(email):
        connection = get_db_connection()
        if not connection:
            return None
        cursor = connection.cursor(dictionary=True)
        try:
            query = "SELECT * FROM users WHERE email = %s"
            cursor.execute(query, (email,))
            user = cursor.fetchone()
            print(f"🔎 User found: {user['username'] if user else 'None'}")
            return user
        except mysql.connector.Error as e:
            print(f"❌ Error fetching user: {e}")
            return None
        finally:
            cursor.close()
            connection.close()


In [22]:
test_user_id = User.create_user("testuser", "testuser@example.com", "password123")
if test_user_id:
    print("Checking user lookup...")
    found_user = User.get_user_by_email("testuser@example.com")
    print(found_user)
else:
    print("User creation failed")


✅ Database connection successful!
✅ User created with id 1
Checking user lookup...
✅ Database connection successful!
🔎 User found: testuser
{'id': 1, 'username': 'testuser', 'email': 'testuser@example.com', 'password_hash': 'scrypt:32768:8:1$TppD4tZbvT8Cjr0s$a214cccd3f77503b67086d4b20e0a9887688d16c1585c195f91efca40489e84fda79693c147df01daad6e84208a6586b40e6645ba47d8329055a0e5ce6cb19aa', 'created_at': datetime.datetime(2025, 9, 30, 23, 49, 9), 'updated_at': datetime.datetime(2025, 9, 30, 23, 49, 9)}


In [16]:
class Design:
    @staticmethod
    def create_design(user_id, room_image, room_name='Untitled Room', style=None):
        connection = get_db_connection()
        if not connection:
            return None
        cursor = connection.cursor()
        try:
            query = """
                INSERT INTO designs (user_id, room_name, room_image, style, created_at, status) VALUES 
                (%s, %s, %s, %s, %s, %s)
            """
            cursor.execute(query, (user_id, room_name, room_image, style, datetime.now(), 'processing'))
            connection.commit()
            print(f"✅ Design created with id {cursor.lastrowid}")
            return cursor.lastrowid
        except mysql.connector.Error as e:
            print(f"❌ Design creation failed: {e}")
            return None
        finally:
            cursor.close()
            connection.close()

    @staticmethod
    def get_user_designs(user_id):
        connection = get_db_connection()
        if not connection:
            return []
        cursor = connection.cursor(dictionary=True)
        try:
            query = """
                SELECT id, room_name, style, status, created_at, room_image, ai_analysis
                FROM designs WHERE user_id = %s ORDER BY created_at DESC
            """
            cursor.execute(query,(user_id,))
            results = cursor.fetchall()
            print(f"🔎 {len(results)} designs found")
            return results
        except mysql.connector.Error as e:
            print(f"❌ Fetching designs failed: {e}")
            return []
        finally:
            cursor.close()
            connection.close()


In [17]:
if test_user_id:
    test_design_id = Design.create_design(test_user_id, "room.jpg", "Living Room", "Modern")
    if test_design_id:
        designs = Design.get_user_designs(test_user_id)
        for d in designs:
            print(d)
    else:
        print("Design creation failed")
else:
    print("User ID not available to create design")


User ID not available to create design
